In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
basedir = './국토교통부_건축물대장_표제부/'
filenames = [f for f in os.listdir(basedir) if f.endswith('.txt')]
filenames

['mart_djy_03_201412.txt',
 'mart_djy_03_201612.txt',
 'mart_djy_03_201712.txt',
 'mart_djy_03_201812.txt',
 'mart_djy_03_201912.txt',
 'mart_djy_03_202012.txt',
 'MART_DJY_TITLE_201512.txt']

In [3]:
colnames_df = pd.read_excel(basedir + '데이터구조.xls', header=1)
print(colnames_df.shape)
colnames_df.head()

(77, 3)


,컬럼 한글명,데이터 타입,비고
0,관리_건축물대장_PK,VARCHAR(33),NaN
1,대장_구분_코드,VARCHAR(1),NaN
2,대장_구분_코드_명,VARCHAR(100),NaN
3,대장_종류_코드,VARCHAR(1),NaN
4,대장_종류_코드_명,VARCHAR(100),NaN


In [4]:
colnames = colnames_df['컬럼 한글명'].tolist()
print(len(colnames))
colnames[:5]

77


['관리_건축물대장_PK', '대장_구분_코드', '대장_구분_코드_명', '대장_종류_코드', '대장_종류_코드_명']

In [5]:
filenames[-2]

'mart_djy_03_202012.txt'

In [6]:
%%time
prac_df = pd.read_csv(basedir + filenames[-2], sep='|', encoding='euc-kr', header=None, names=colnames)
print(prac_df.shape)
prac_df.head()

<string>:2: DtypeWarning: Columns (15,31,37,58,59,60,61,62,67) have mixed types. Specify dtype option on import or set low_memory=False.


(7837517, 77)
Wall time: 1min 11s


,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,내진_설계_적용_여부,내진_능력
0,27710-2503,1,일반,2,일반건축물,대구광역시 달성군 화원읍 천내리 387-5번지,대구광역시 달성군 화원읍 비슬로512길 57-9,NaN,27710,25021.0,0.0,387,5,NaN,NaN,NaN,0,2.771042e+11,25001.0,0.0,57.0,9.0,NaN,1.0,부속건축물,0.0,2.22,0.00,2.22,2.22,0.00,99,기타구조,샌드위치판넬,04000,제2종근린생활시설,방앗간,90,기타지붕,판넬,0,1,0.0,1,0,0,0,1,2.22,2.22,0,0.0,0,0.0,0,0.0,0,0.0,19830325,NaN,1.98303e+07,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110414,NaN,NaN
1,28710-6007,1,일반,2,일반건축물,인천광역시 강화군 양도면 도장리 685-1번지,인천광역시 강화군 양도면 강화남로428번길 46-20,NaN,28710,35027.0,0.0,685,1,NaN,NaN,NaN,0,2.871043e+11,35001.0,0.0,46.0,20.0,NaN,0.0,주건축물,522.0,98.94,18.95,141.60,141.60,27.12,11,벽돌구조,조적조,01000,단독주택,주택,10,(철근)콘크리트,슬라브,0,1,6.6,2,0,0,0,0,0.00,141.60,0,0.0,0,0.0,0,0.0,0,0.0,19960604,1.99606e+07,1.99805e+07,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110414,NaN,NaN
2,28710-23420,1,일반,2,일반건축물,인천광역시 강화군 선원면 냉정리 916-1번지,인천광역시 강화군 선원면 중앙로 468,NaN,28710,31027.0,0.0,916,1,NaN,NaN,NaN,0,2.871032e+11,31002.0,0.0,468.0,0.0,NaN,0.0,주건축물,660.0,171.69,26.01,171.69,171.69,26.01,11,벽돌구조,"조적조, 경량철골조",01000,단독주택,"단독주택, 창고",90,기타지붕,판넬,0,1,4.2,1,0,0,0,1,72.60,99.09,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,2.00206e+07,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110414,NaN,NaN
3,28710-14164,1,일반,2,일반건축물,인천광역시 강화군 송해면 하도리 277-6번지,인천광역시 강화군 송해면 강화대로 819,NaN,28710,39026.0,0.0,277,6,NaN,NaN,NaN,0,2.871022e+11,39003.0,0.0,819.0,0.0,NaN,0.0,주건축물,226.0,70.03,30.99,70.03,70.03,30.99,32,경량철골구조,경량철골조,01000,단독주택,단독주택,90,기타지붕,판넬,0,1,4.0,1,0,0,0,0,0.00,70.03,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,2.00203e+07,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110414,NaN,NaN
4,28720-925,1,일반,2,일반건축물,인천광역시 옹진군 북도면 신도리 500번지,인천광역시 옹진군 북도면 신도로896번길 28,NaN,28720,31022.0,0.0,500,0,NaN,NaN,NaN,0,2.872043e+11,31001.0,0.0,28.0,0.0,NaN,0.0,주건축물,386.0,51.30,13.29,51.30,51.30,13.29,12,블록구조,블록구조,01000,단독주택,단독주택,30,슬레이트,슬레이트,0,1,0.0,1,0,0,0,0,0.00,51.30,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,1.97712e+07,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110414,NaN,NaN


In [12]:
def apply_int(x):
    # x is a number
    
    if pd.isna(x) == True:
        return x
    else:
        if x == '`':
            return 1
        else:
            return int(x)

In [13]:
prac_df['구조_코드'] = prac_df['구조_코드'].apply(apply_int).astype('Int64')

In [25]:
prac_df['구조_코드_명'].value_counts()

철근콘크리트구조         1927346
벽돌구조             1726427
일반목구조            1363582
경량철골구조            841587
블록구조              765199
일반철골구조            677872
기타조적구조            200204
강파이프구조            176304
기타강구조              26144
조적구조               21501
철골철근콘크리트구조         18230
석구조                16512
기타구조               10976
철골콘크리트구조           10803
목구조                 6732
통나무구조               4446
프리케스트콘크리트구조         3987
컨테이너조               3743
조립식판넬조              2465
기타콘크리트구조            2456
시멘트블럭조              2410
철골구조                1878
콘크리트구조              1392
철파이프조                600
트러스목구조               508
공업화박판강구조(PEB)        477
기타철골철근콘크리트구조         235
철골철근콘크리트합성구조         159
스틸하우스조               150
흙벽돌조                 140
막구조                   57
단일형강구조                53
돌담 및 토담조              53
트러스구조                 44
라멘조                   34
보강콘크리트조               13
석회 및 흙혼합 벽돌조           6
Name: 구조_코드_명, dtype: int64

In [26]:
prac_df['구조_코드'].value_counts()

21    1927346
11    1726427
51    1363582
32     841587
12     765199
31     677872
19     200204
33     176304
39      26144
10      21501
42      18204
13      16512
99      10976
41      10803
50       6732
52       4446
22       3987
74       3743
63       2465
29       2456
61       2410
30       1878
20       1392
23        600
53        508
34        477
49        235
43        159
72        140
14         97
81         57
24         53
35         53
37         53
36         44
26         34
40         26
17         13
27          6
Name: 구조_코드, dtype: int64

In [16]:
print(prac_df.shape)
prac_df.dropna(subset=['구조_코드_명', '구조_코드'], inplace=True)
prac_df.shape

(7837517, 77)


(7814725, 77)

In [27]:
prac_df['구조_코드_명'].nunique()

37

In [28]:
prac_df['구조_코드'].nunique()

39

In [29]:
for structure in tqdm(prac_df['구조_코드_명'].unique()):
    structure_df = prac_df[prac_df['구조_코드_명'] == structure]
    print(structure)
    print(structure_df['구조_코드'].unique())

  3%|██▏                                                                                | 1/37 [00:00<00:12,  2.86it/s]

기타구조
<IntegerArray>
[99]
Length: 1, dtype: Int64


  5%|████▍                                                                              | 2/37 [00:01<00:27,  1.27it/s]

벽돌구조
<IntegerArray>
[11]
Length: 1, dtype: Int64


  8%|██████▋                                                                            | 3/37 [00:02<00:30,  1.11it/s]

...

막구조
<IntegerArray>
[81]
Length: 1, dtype: Int64


 97%|███████████████████████████████████████████████████████████████████████████████▊  | 36/37 [00:15<00:00,  3.83it/s]

석회 및 흙혼합 벽돌조
<IntegerArray>
[27]
Length: 1, dtype: Int64


100%|██████████████████████████████████████████████████████████████████████████████████| 37/37 [00:15<00:00,  2.38it/s]

보강콘크리트조
<IntegerArray>
[17]
Length: 1, dtype: Int64
